# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Tuesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [1]:
# Your code here

import requests
import pandas as pd
import time

# Your API key
api_key = "OSi7PPNtPI3r2KsQa1OKU8hWTL5WoSO788SpEnCW"

# API request headers
headers = {
    "Accept": "application/json",
    "x-api-key": api_key
}

# Search query
query = "data science"

# API endpoint
api_endpoint = "https://api.semanticscholar.org/graph/v1/paper/search"

# Request parameters
params = {
    "query": query,
    "fields": "title,abstract",
    "limit": 100  # Max results per request
}

# List to store all collected papers
paper = []

# Loop to fetch batches of results
for offset in range(0, 10000, 100):
    params["offset"] = offset
    response = requests.get(api_endpoint, headers=headers, params=params)

    if response.status_code == 200:
        papers = response.json().get('data', [])
        if papers:
            paper.extend(papers)
            print(f"Fetching abstracts from paper {offset}.")
        else:
            print(f"No papers found for offset {offset}.")
            break
    elif response.status_code == 429:
        print(f"Rate limit exceeded. Waiting 1 second for offset {offset}.")
        time.sleep(1)  # Waiting time
        response = requests.get(api_endpoint, headers=headers, params=params)
        if response.status_code == 200:
            papers = response.json().get('data', [])
            if papers:
                paper.extend(papers)
                print(f"Collected papers after retry for offset {offset}.")
            else:
                print(f"No papers found after retry for offset {offset}.")
                break
        else:
            print(f"Error after retrying: {response.status_code}, Offset: {offset}")
            break
    else:
        print(f"Error fetching data: {response.status_code}, Offset: {offset}")
        break

    time.sleep(1)

# Extract titles and abstracts
abstracts = [(paper["title"], paper.get("abstract", "N/A")) for paper in paper]

# Save to CSV file
output_file = "data_science_abstracts.csv"
df = pd.DataFrame(abstracts, columns=["Title", "Abstract"])
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Fetched {len(abstracts)} abstracts. Saved to {output_file}.")

Fetching abstracts from paper 0.
Fetching abstracts from paper 100.
Fetching abstracts from paper 200.
Fetching abstracts from paper 300.
Fetching abstracts from paper 400.
Fetching abstracts from paper 500.
Fetching abstracts from paper 600.
Fetching abstracts from paper 700.
Fetching abstracts from paper 800.
Fetching abstracts from paper 900.
Error fetching data: 400, Offset: 1000
Fetched 1000 abstracts. Saved to data_science_abstracts.csv.


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [3]:
# Write code for each of the sub parts with proper comments.
!pip install pandas nltk
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load the CSV file
data = pd.read_csv('data_science_abstracts.csv')

# Initialize the stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Define the stopwords
stop_words = set(stopwords.words('english'))

# Function to clean the text
def clean_text(text):
    if not isinstance(text, str):
        return ''  # Return an empty string if the input is not a valid string

    # 1. Remove special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    # 2. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 3. Remove stopwords
    words = nltk.word_tokenize(text)
    words = [word for word in words if word.lower() not in stop_words]

    # 4. Convert to lowercase
    words = [word.lower() for word in words]

    # 5. Apply stemming
    stemmed_words = [stemmer.stem(word) for word in words]

    # 6. Apply lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]

    return ' '.join(lemmatized_words)

# Convert the 'Abstract' column to strings, filling NaN values with an empty string
data['Abstract'] = data['Abstract'].astype(str).fillna('')

# Apply the cleaning function to the Abstract column
data['Cleaned_Abstract'] = data['Abstract'].apply(clean_text)
print("Cleaned saved to 'cleaned_data_science_abstracts.csv")
# Save the cleaned data into a new CSV file
data.to_csv('cleaned_data_science_abstracts.csv', index=False)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Cleaned data has been saved to 'cleaned_data_science_abstracts.csv


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [4]:
# Your code here

import pandas as pd
import nltk
import spacy
from nltk import pos_tag, word_tokenize
from nltk.chunk import ne_chunk
from nltk.tree import Tree
from collections import Counter

# Download necessary NLTK resources (uncomment if you haven't already)
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Load the cleaned CSV file
file = "cleaned_data_science_abstracts.csv"
data = pd.read_csv(file)

# Load the spaCy model for NER and dependency parsing
nlp = spacy.load("en_core_web_sm")

# Function to perform POS tagging and count POS types
def pos_analysis(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)  # POS tagging

    # Counting specific POS tags
    counts = Counter(tag for word, tag in tagged)
    noun_count = sum(counts[tag] for tag in ['NN', 'NNS', 'NNP', 'NNPS'])
    verb_count = sum(counts[tag] for tag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'])
    adjective_count = sum(counts[tag] for tag in ['JJ', 'JJR', 'JJS'])
    adverb_count = sum(counts[tag] for tag in ['RB', 'RBR', 'RBS'])

    pos_counts = {
        'Nouns': noun_count,
        'Verbs': verb_count,
        'Adjectives': adjective_count,
        'Adverbs': adverb_count
    }

    return tagged, pos_counts

# Function to perform dependency parsing using spaCy
def dependency_parsing(text):
    doc = nlp(text)
    return [(token.text, token.dep_, token.head.text) for token in doc]

# Function to perform constituency parsing using nltk
def constituency_parsing(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    chunked = ne_chunk(tagged)

    def tree_to_string(tree):
        """ Convert an NLTK tree to a string to visualize the tree structure. """
        if isinstance(tree, Tree):
            return f"({tree.label()} {' '.join(tree_to_string(child) for child in tree)})"
        else:
            return tree[0]

    return tree_to_string(chunked)

# Function to extract named entities and their counts
def ner_analysis(text):
    doc = nlp(text)
    entities = Counter(ent.label_ for ent in doc.ents)
    return entities, [(ent.text, ent.label_) for ent in doc.ents]

# Perform analysis for each abstract and store results
results = []

for index, row in data.iterrows():
    abstract = row['Cleaned_Abstract']

    # Skip empty abstracts
    if pd.isna(abstract) or abstract.strip() == "":
        continue

    # POS Analysis
    tagged, pos_counts = pos_analysis(abstract)

    # Dependency Parsing
    dependency_parse = dependency_parsing(abstract)

    # Constituency Parsing
    constituency_parse = constituency_parsing(abstract)

    # NER Analysis
    ner_counts, ner_entities = ner_analysis(abstract)

    results.append({
        'Abstract': abstract,
        'POS Tagged': tagged,
        'POS Counts': pos_counts,
        'Dependency Parsing': dependency_parse,
        'Constituency Parsing': constituency_parse,
        'Named Entities': ner_entities,
        'NER Counts': ner_counts
    })

# Save the results to a new CSV file
results_df = pd.DataFrame(results)
results_df.to_csv("analyzed_data_science_abstracts.csv", index=False)
print("Data is saved in analyzed_data_science_abstracts.csv")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


Data is saved in analyzed_data_science_abstracts.csv


#**Comment**
Make sure to submit the cleaned data CSV in the comment section - 10 points

[cleaned_data_science_abstracts.csv](https://myunt-my.sharepoint.com/:x:/g/personal/sivarohithjampana_my_unt_edu/EbN1l-sgbZ1EnynIgvXzX8ABhZmp_LgtvSAGmL91u6pYRA?e=Molmge)


[analyzed_data_science_abstracts](https://myunt-my.sharepoint.com/:x:/g/personal/sivarohithjampana_my_unt_edu/EZDO4mbA84REpw_6ZzzidBEBphe9YSgtXE6s3Dvm48BaAA?e=XVfQ4E)

# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [11]:
# Write your response below
'''The assignment was a good learning experience for webscraping and data analysis.I got experience of using API.I enjoyed applying NLP techniques like POS tagging and Named Entity Recognition, finding insights from the data. The allocated time was reasonable, allowing thorough exploration while feeling slightly rushed on parsing nuances.'''

'The assignment was a good learning experience for webscraping and data analysis.I got experience of using API.I enjoyed applying NLP techniques like POS tagging and Named Entity Recognition, finding insights from the data. The allocated time was reasonable, allowing thorough exploration while feeling slightly rushed on parsing nuances.'